In [86]:
from IPython.display import HTML, display, SVG
display(HTML('''
 <link rel="stylesheet" href="custom.css" type="text/css" />
'''))

# From SCADA to IoT with Python

## Nahuel Defossé (D3f0)



<img src="img/scipy.png" style="float: right">


# Disclaimer
 * Not a good English speaker but definetly worse at Protuguese.
 * Some material cannot be distributed openly, approach me personaly to get more details.

# About me <i class="fa fa-info-circle" aria-hidden="true">&nbsp;</i>
 * Member of Research Group focused on Micro-Controllers and Networks.
 * Chief of Practical Assgnments at Universidad Nacional de la Patagonia
 * Python Advocate
     - Python courses in my local University
     - Frequently give talks at [Patagonian Python Meetup](http://www.meetup.com/es/Patagonia-Python-Meetup/)
     - Chair at SciPy Latinamerica [2013](http://www.scipyla.org/conf/2013/) and [2014](http://www.scipyla.org/conf/2014/)
 * Worked in Electric Transportation industry for 3 years
 
<h3>
<a href="https://twitter.com/D3f0"><i class="fa fa-twitter" aria-hidden="true">&nbsp;</i></a>
<a href="https://github.com/D3f0"><i class="fa fa-github" aria-hidden="true">&nbsp;</i></a>
</h3>

# Owed credit <i class="fa fa-users" aria-hidden="true">&nbsp;</i>

 * Ricardo A **Lopez** (UNPSJB, Argentina)
 * Fernando A **Tinetti**(UNLP, Argentina)
 * Lautaro **Pecile** (UNPSJB, Argentina)
 * Diego **van Haaster** (UNPSJB, Argentina)
 * Sebastián **Wahler** (UNPSJB, Argentina)
 * Fernando **Páez** (UNPSJB, Argentina)
 * Manuel **Urriza** (UNS, Argentina)

# Talk organization

* Python for Process Monitoring (Construct, Twisted, Django)
* Python as tooling (PlatformIO)
* Python in the MicroController
* Conclusions
* Future work

<!-- # SCADA <i class="fa fa-tachometer" aria-hidden="true">&nbsp;</i> -->
# Python for Process Monitoring (with Web UI)

### Applied to power station


# General Requirements

* Noise Resilience to High noise enviroments.
* Integrate with old Power Meters
* Interactive web HMI
* Reports
* Low cost

<div style="text-align: center">
<img style="width: 60%;" src="img/estructura.png">
</div>

<div style="text-align: center">
<img style="width: 70%;" src="img/estructura2.png">
</div>

### IED

<div>
<img style="float: left;" src="img/alpha2.png"/>
<img style="float: right;" src="img/ied.png"/>
</div>

# COMaster

<img style="width: 50%" src="img/comaster.png">

# Wire protocol

<pre style="font-size: .7em;">
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┐┌─────┐┌─────┬─────┐
│ SOF │ SEQ │ QTY │ SRC │ DST │ CMD │ CMD ││ ... ││ BCL │ BCH │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┘└─────┘└─────┴─────┘
┌─────┬───────────┬───────────┬─────┬───────────┬───────────┐  
│ QSV │ SV0       │ SV1       │ QDI │ DI0       │ DI1       │  
└─────┴───────────┴───────────┴─────┴───────────┴───────────┘  
┌─────┬───────────┬───────────┬─────┬─────────────────────────┐
│ QAI │ AI0       │ AI1       │ QEV │ EV..                    .
└─────┴───────────┴───────────┴─────┴─────────────────────────┘
</pre>

# A quick peek at production logs

* http://localhost:19001/logtail/poll_mara



# Parsing Mara Frame 

 * `pip install construct`
 * Forget about Python built in `struct`, Constrct gives a DSL for binary parsing.

In [64]:
from construct import *

PascalString = Struct("PascalString",
    UBInt8("length"),
    Bytes("data", lambda ctx: ctx.length),
)
PascalString.parse(b"\x05helloXXX")

Container({'length': 5, 'data': b'hello'})

# [Mara Definition]()

In [67]:
MaraFrame = BaseMaraStruct(
    'Mara',
    ULInt8('sof'),
    ULInt8('length'),
    ULInt8('dest'),
    ULInt8('source'),
    ULInt8('sequence'),
    ULInt8('command'),
    # Probe(),

    If(lambda ctx: ctx.command == 0x10,
       # Probe(),
       Optional(Payload_10),
    ),

    If(lambda ctx: ctx.command == 0x12,
       PEHAdapter(Payload_PEH),
    ),
    ULInt16('bcc')
)

NameError: name 'BaseMaraStruct' is not defined

# Data Storage

* Initially written with SQLAlchemy and Flask
* Migrated to Django due to ORM, Admin, REST 3rd party apps and management commands.

<img src="img/mara.svg">

# [Saving data on DB](https://github.com/D3f0/txscada/blob/master/src/pysmve/nguru/apps/mara/models.py#L200)

In [68]:
# DI
for value, di in zip(iterbits(payload.dis, length=16), self.dis):
    # Update value takes care of maskinv
    di.update_value(value, q=0, last_update=timestamp)
    di_count += 1
# AI
for value, ai in zip(payload.ais, self.ais):
    value = value & 0x0FFF
    q = (value & 0xC000) >> 14
    ai.update_value(value, last_update=timestamp, q=q)
    ai_count += 1
# ...

NameError: name 'iterbits' is not defined

# Human Machine Interface (HMI)



In [71]:
SVG('''
<svg  xmlns="http://www.w3.org/2000/svg"
      xmlns:xlink="http://www.w3.org/1999/xlink">
    <rect x="10" y="10" height="100" width="100"
          style="stroke:#F00; fill: #0F0"/>
</svg>
''')

![](img/smve1.png)

# SVG

* Each *mutable* element has an ID **SVGElement** and belongs to a **Screen** (SVG file).
* A **Screen** can have a parent and might have links to other screens (tree structure).
* Each **SVGElement** SVG attribute has a formula associated. 
* When a new value is read from the field, the formula is calculated and the property refreshed on screen.
* This formulas are a Pythonish excel syntax, and are loaded into the DB with Pandas' [`read_excel`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)

<img src="img/hmi.svg">

# Formula

```python
SI(di.E2152B04.value,si(int(eg.E2189S04.text),eg.E2189S04.stroke,12),si(int(eg.E2152B04.text),6,10))
```


# Historical data
<img src="img/curvas.png">

### Would be great to rewrite this in Bokeh

# Towards a IoT scenario

* Bigger and more deiverse enviroments
* **Arduino API** compatible hardware 


# PlatformIO

<center>
<img src="img/platformio-logo.png">
</center>

 * Colourful command-line output
 * IDE Integration with Arduino, Atom, CLion, Eclipse, Emacs, Energia, Qt Creator, Sublime Text, Vim, Visual Studio
 * Cloud compiling and Continuous Integration with AppVeyor, Circle CI, Drone, Shippable, Travis CI
 * Built-in Serial Port Monitor and configurable build -flags/-options
 * Automatic firmware uploading
 * Pre-built toolchains, frameworks for the popular Hardware Platforms

# Supported PlatformIO boards

<img src="img/plataformas.png">

<div>
    <img src="img/pio.gif">
</div>

# MQTT

* Message Oriented Middleware OASIS Standard
* Very efficient and desinged for constrained hardware
* A message has:
    - Topic
    - Payload
* FOSS and Comercial implementation
* Used by Facebook at [Messenger](https://www.facebook.com/notes/facebook-engineering/building-facebook-messenger/10150259350998920/)

<img src="img/mqtt_msft.jpg">
Image from <a href="http://pt.slideshare.net/paolopat/mqtt-iot-protocols-comparison/8">Paolo Patierno</a>

# An MQTT Example

* Built in PlatformIO IDE
* Arduino callback


In [80]:
!mosquitto_pub -t /esp/led -m 1

In [82]:
!mosquitto_pub -t /esp/led -m 0

# Python in the Microncotoller

<img src="img/micropython.jpeg">

* Origninally targeted to PyBoard
    - LEDs
    - Pins and GPIO
    - External interrupts
    - Timers
    - PWM (pulse width modulation), ADC (analog to digital conversion), DAC (digital to analog conversion), UART (serial bus), SPI bus, I2C bus
* Python 3
* Have support for many other Boards 



# Micro MicroPython port on EDUCIAA

* EDU-CIAA-NXP is de educational version of the Argentinan Open Industrial Computer ([CIAA](http://www.proyecto-ciaa.com.ar/))
* More on [this video](https://www.youtube.com/watch?v=3M9xytUpySk)
* Demo!

# Future Work

* Kivy native app for monitoring processes
* Integration with QR Codes
* Expose Django DB for analytics through Pandas
* Simplify deployment with docker containers

# Conclusions so far

* Django ORM might not be the best way store data
* Not loosing any package has to be considered in the wire protocol, MQTT helps
* Design the system for fault tolerance in every level of abstraction
* Waiting MQTT support for micropython on ESP8266 based boards

# Questions?

# Thanks

Take a look at &nbsp;<span style="font-size: 3em">
<a href="https://github.com/D3f0/txscada"><i class="fa fa-github" aria-hidden="true">&nbsp;</i></a>
</span>




In [88]:
!git add img && git add -u && git commit -m "Updates $(date)" && git push origin master

[master 1376ebc] Updates Fri May 20 14:37:36 ART 2016
 1 file changed, 44 insertions(+), 49 deletions(-)
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.18 KiB | 0 bytes/s, done.
Total 6 (delta 2), reused 0 (delta 0)
To git@github.com:D3f0/talks.git
   8bcae58..1376ebc  master -> master


In [75]:
!open -a iTerm .